In [1]:
import os
import datetime
from dateutil.relativedelta import relativedelta
from unittest import result
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from functools import reduce
from cuml.experimental.preprocessing import MinMaxScaler
from cuml.preprocessing import Normalizer
import pandas as pd
import pickle

import cudf
import cuml

import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [2]:
INPUT_DIR = './h-and-m-personalized-fashion-recommendations/'
transactions = pd.read_csv(INPUT_DIR + 'transactions_train.csv')
customers = pd.read_csv(INPUT_DIR + 'customers.csv')
articles = pd.read_csv(INPUT_DIR + 'articles.csv')
sub = pd.read_csv(INPUT_DIR + 'sample_submission.csv')

In [3]:
ALL_CUSTOMER = customers['customer_id'].unique().tolist()
ALL_ARTICLE = articles['article_id'].unique().tolist()

customer_ids = dict(list(enumerate(ALL_CUSTOMER)))
article_ids = dict(list(enumerate(ALL_ARTICLE)))

customer_map = {u: uidx for uidx, u in customer_ids.items()}
article_map = {i: iidx for iidx, i in article_ids.items()}

articles['article_id'] = articles['article_id'].map(article_map).astype('int32')
customers['customer_id'] = customers['customer_id'].map(customer_map).astype('int32')
transactions['article_id'] = transactions['article_id'].map(article_map).astype('int32')
transactions['customer_id'] = transactions['customer_id'].map(customer_map).astype('int32')
transactions['price'] = transactions['price'].astype('float32')
transactions['sales_channel_id'] = transactions['sales_channel_id'].astype('int8')

sub['customer_id'] = sub['customer_id'].map(customer_map)
# first_week_sales_pred['article_id'] = first_week_sales_pred['article_id'].map(article_map) 

In [4]:
transactions.dtypes

t_dat                object
customer_id           int32
article_id            int32
price               float32
sales_channel_id       int8
dtype: object

In [5]:
transactions['customer_id'].max()

1371979

In [6]:
transactions['article_id'].max()

105539

In [7]:
transactions['customer_id'].max()

1371979

In [8]:
transactions['article_id'].max()

105539

In [9]:
import pickle

In [10]:
target_path='dataset/'

In [11]:
with open(target_path + "customer_ids.pickle", 'wb') as handle:
    pickle.dump(customer_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with open(target_path + "article_ids.pickle", 'wb') as handle:
    pickle.dump(customer_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
transactions.to_parquet(target_path + "transactions.parquet")
articles.to_parquet(target_path + "articles.parquet")
customers.to_parquet(target_path + "customers.parquet")
sub.to_parquet(target_path + "submission.parquet")